In [1]:
print("ay")
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy.stats import norm
import probfit
import iminuit
import matplotlib.mlab as mlab
from scipy.optimize import curve_fit
from numpy import random, exp
from scipy.stats import crystalball
from pylab import *
from scipy.special import erf
from scipy.stats import norm
from scipy.optimize import leastsq

ay


In [2]:
original_df1=pd.read_hdf("/mnt/t3nfs01/data01/shome/micheli/energyRegProject_20180223/CMSSW_8_0_4/src/RegressionTraining/applyRegression/ntuples_NN/Ntup_10Nov_Photon_testing.hd5")
#original_df_EE=pd.read_hdf("/mnt/t3nfs01/data01/shome/micheli/energyRegProject_20180223/CMSSW_8_0_4/src/RegressionTraining/applyRegression/ntuples_NN_EE/Ntup_10Nov_Photon_testing_EE.hd5")
print("finished")

Opening /mnt/t3nfs01/data01/shome/micheli/energyRegProject_20180223/CMSSW_8_0_4/src/RegressionTraining/applyRegression/ntuples_NN/Ntup_10Nov_Photon_testing.hd5 in read-only mode
finished


In [ ]:
#todo: Add errors in histogramm and ratio sqrt(entries)
k=0
#for original_df in [original_df1,original_df_EE]:
for df in [original_df1,]:#original_df_EE

    if k==0:
        suffix='NN'
        k+=1
    else:
        suffix='NN_EE'
    for column in ['pt','scEta','rhoValue']: 
        df=original_df1.copy()
        df=df[df['response']>.95]
        df=df[df['response']<1.02]

        df=df[np.abs(df[column]-df[column].median()) <= (2*df[column].std())]#outlier detection

        df.loc[:,'response']=np.reciprocal(df.loc[:,'response'].values) #invert response

        x_plot=[]
        y_plot=[]
        y_err=[]
        y_err_fit=[]
        y_err_sigma=[]
        notused,bins=pd.cut(df[column],20,retbins=True) #equidistant bins 
        for i in xrange(0,len(bins)-1):
            print(i)
            df_temp=df[df[column] >=bins[i]]
            df_temp=df_temp[df_temp[column]<=bins[i+1]]
            if df_temp.size==0:
                print i
                print "empty"
                continue

            ax=plt.subplot2grid((6,3),(0,0), rowspan=4,colspan=4)    
            bound = (.95, 1.03)
            data=df_temp.loc[:,'response'].values
            print "slow="
            normalized_crystalball = probfit.Normalized(probfit.crystalball, bound)
            print "1"
            unbinned_likelihood = probfit.UnbinnedLH(normalized_crystalball, data)
            
            #
            #best_try = probfit.try_uml(normalized_crystalball, data, **params)
            print("this")
            minuit = iminuit.Minuit(unbinned_likelihood,pedantic=False, alpha=.5,limit_alpha=(0.5,0.7),n=3., mean=1.01,limit_mean=(0.99,1.02),sigma=0.01,limit_sigma=(0.001,0.011))
            print("that")
            minuit.migrad()
            print "fuk"

            coeff=minuit.args


            y_plot.append(coeff[2])
            x_plot.append(df_temp[column].values[np.argmin(df_temp['response'].values-coeff[2])])
            y_err_fit.append(minuit.errors[2])
            y_err_sigma.append(coeff[3])
            (data_edges, datay), (errorp, errorm), (total_pdf_x, total_pdf_y),parts=unbinned_likelihood.draw(minuit,nfbins=100,parmloc=((0.7,0.95)))

            plt.title("hist $response$ for ${}$ in {} to {}".format(column,int(10*bins[i])/10.,int(10*bins[i+1])/10.))
            plt.subplot2grid((5,3),(4,0),rowspan=2,colspan=4).set_yticks([0.8,1])
            ratio=datay/total_pdf_y
            plt.ylim(0.8,1.2)
            plt.plot((data_edges[1:]+data_edges[:-1])/2,ratio,'bo')
            #unbinned_likelihood.draw_residual(minuit,parmloc=(0.7,0.95))
            plt.ylabel(r"$ratio$")
            plt.xlabel("response")
            plt.savefig("plots/CB_{}/{}_{}".format(column,suffix,i)) 
            plt.show()
            #minuit = iminuit.Minuit(unbinned_likelihood, **best_try)
            #unbinned_likelihood.show(minuit)


        plt.figure()
        plt.title("Profile $response$ vs ${}$, Fit error".format(column))
        plt.ylabel(r"$response$")
        plt.xlabel(r"${}$".format(column))
        plt.errorbar(x_plot,y_plot,y_err_fit,fmt='b.')
        plt.savefig("plots/profile_plots_response/Fit_Error_{_}response_vs_{}".format(suffix,column))
        plt.show()
        plt.close()
        plt.figure()
        plt.title("Profile $response$ vs ${}$, Sigma error".format(column))
        plt.ylabel(r"$response$")
        plt.xlabel(r"${}$".format(column))
        plt.errorbar(x_plot,y_plot,y_err_fit,fmt='b.')
        plt.savefig("plots/profile_plots_response/Sigma_Error_{_}response_vs_{}".format(suffix,column))
        plt.show()
        plt.close()


0


In [ ]:
unbinned_likelihood.draw_residual()
unbinned_likelihood.show()